In [2]:
import pandas as pd

In [89]:
# train_df = pd.read_csv("data/fake_news_dataset_2.csv")
# test_df = pd.read_csv("data/fake_news_test_dataset_2.csv")
# val_df = pd.read_csv("data/fake_news_validation_dataset_2.csv")
# df = pd.concat([train_df, test_df, val_df], ignore_index=True)
# df.drop("Unnamed: 0", axis=1, inplace=True)
# # df.drop_duplicates(subset=["title"], keep="last", inplace=True)
# df = df.sample(frac=1)
# df.reset_index(inplace=True)
# len(df)

In [4]:
true = pd.read_csv("../data/True.csv")
fake = pd.read_csv("../data/Fake.csv")
true["label"] = 1
fake["label"] = 0
df = pd.concat([true, fake], ignore_index=True)
# df.drop_duplicates(subset=["title"], keep="last", inplace=True)
df = df.sample(frac=1)
df.reset_index(inplace=True)
len(df)

44898

In [5]:
import re


def clean_text(text):
  text = text.lower().strip()
  text = re.sub(r"https?://\S+|www\.\S+", "", text)
  text = re.sub(r"\[.*?\]", "", text)
  text = re.sub(r"\w*\d\w*", "", text)
  text = re.sub(r"<.*?>+", "", text)
  text = re.sub(r"\n", " ", text)
  text = re.sub(r"\s+", " ", text)
  text = re.sub(r"\W", " ", text)
  return text


In [6]:
df["text"] = df["text"].apply(clean_text)

In [7]:
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(
  df["text"], df["label"], test_size=0.25
)

In [94]:
from sklearn.feature_extraction.text import TfidfVectorizer

vect = TfidfVectorizer()
X_train = vect.fit_transform(X_train)
X_test = vect.transform(X_test)

In [95]:
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import classification_report

model = DecisionTreeClassifier()
model.fit(X_train, y_train)
y_pred = model.predict(X_test)

print(classification_report(y_test, y_pred))
print(model.score(X_test, y_test))


              precision    recall  f1-score   support

           0       0.99      1.00      1.00      5788
           1       1.00      0.99      1.00      5437

    accuracy                           1.00     11225
   macro avg       1.00      1.00      1.00     11225
weighted avg       1.00      1.00      1.00     11225

0.9961692650334075


In [96]:
model = RandomForestClassifier()
model.fit(X_train, y_train)
y_pred = model.predict(X_test)

print(classification_report(y_test, y_pred))
print(model.score(X_test, y_test))

              precision    recall  f1-score   support

           0       0.99      0.99      0.99      5788
           1       0.99      0.99      0.99      5437

    accuracy                           0.99     11225
   macro avg       0.99      0.99      0.99     11225
weighted avg       0.99      0.99      0.99     11225

0.9889532293986637


In [ ]:
# Good classification models
from sklearn.ensemble import GradientBoostingClassifier

model = GradientBoostingClassifier()
model.fit(X_train, y_train)
y_pred = model.predict(X_test)

print(classification_report(y_test, y_pred))
print(model.score(X_test, y_test))

              precision    recall  f1-score   support

           0       1.00      1.00      1.00      5788
           1       1.00      1.00      1.00      5437

    accuracy                           1.00     11225
   macro avg       1.00      1.00      1.00     11225
weighted avg       1.00      1.00      1.00     11225

0.9964365256124722


In [98]:
# save the model and vectorizer
import pickle

with open("gradient_boosting_classifier.pkl", "wb") as f:
  pickle.dump(model, f)
with open("tfidf_vectorizer.pkl", "wb") as f:
  pickle.dump(vect, f)